### Health Check

In [3]:
$Info = Get-ComputerInfo 

$computerInfoProperties = $Info | Select-Object -Property OsLastBootUpTime,BiosReleaseDate,TimeZone,WindowsInstallDateFromRegistry,
             OsName,OsOperatingSystemSKU,OSArchitecture,WindowsVersion,WindowsBuildLabEx,OsMuiLanguages,BiosCaption,CsBootupState,CsDNSHostName,CsDomain,CsProcessors,OsSystemDirectory

$additionalDetails = [ordered]@{                                                                              # Calculating additional system details
    'TimeZone' = (Get-TimeZone).Id
    'Current System Time' = Get-Date
    'PhysicalHostName' = (Get-Item "HKLM:\SOFTWARE\Microsoft\Virtual Machine\Guest\Parameters").GetValue("PhysicalHostName")
    'NW Adapters' = {$Info.CsNetworkAdapters.Description -join ', '}.Invoke()
    'IPAddress' = {$Info.CsNetworkAdapters.IPAddresses -join ', '}.Invoke()

    'Cpu Usage Percentage' = (Get-CimInstance win32_processor | Measure-Object LoadPercentage -Average).Average
    'Memory Usage Percentage' = { [Math]::Ceiling(($Info.OsTotalVisibleMemorySize - $Info.OsFreePhysicalMemory) * 100 / $Info.OsTotalVisibleMemorySize) }.Invoke()
    'C Drive Percentage Free' = { $disk = Get-CimInstance Win32_LogicalDisk -Filter "DeviceID='C:'"; [Math]::Ceiling(($disk.FreeSpace / $disk.Size) * 100) }.Invoke()
}

$computerInfoList = $computerInfoProperties.PSObject.Properties | ForEach-Object { @{ $_.Name = $_.Value } }  # Converting properties to key-value pairs
$combinedData = $computerInfoList + $additionalDetails                                                        # Combining both sets of data into one list
$combinedData | Format-Table -HideTableHeaders -AutoSize                                                      # Displaying the combined list


OsLastBootUpTime               12/3/2023 11:09:56 AM
BiosReleaseDate                7/11/2023 7:00:00 PM
TimeZone                       (UTC-06:00) Central Time (US & Canada)
WindowsInstallDateFromRegistry 3/7/2023 12:48:05 AM
OsName                         Microsoft Windows 11 Enterprise
OsOperatingSystemSKU           EnterpriseEdition
OsArchitecture                 ARM 64-bit Processor
WindowsVersion                 2009
WindowsBuildLabEx              22621.1.arm64fre.ni_release.220506-1250
OsMuiLanguages                 {en-US}
BiosCaption                    Hyper-V UEFI Release v4.1
CsBootupState                  Normal boot
CsDNSHostName                  W11ARM
CsDomain                       WORKGROUP
CsProcessors                   {Ampere(R) Altra(R) Processor}
OsSystemDirectory              C:\Windows\system32
TimeZone                       Central Standard Time
Current System Time            12/3/2023 8:45:22 PM
PhysicalHostName               CHI231053017064
NW Adapters       

This shows which processes are consuming the most cpu, memory, network and disk and have the most number of handles and threads currently on the system.

In [8]:
#$Host.UI.RawUI.BufferSize = New-Object Management.Automation.Host.Size(127, $Host.UI.RawUI.BufferSize.Height)  #v2: One line equivalent

$objArr = Get-Process | Select-Object @{Name='Process'; Expression={$_.ProcessName+'►'+$_.Id}}, CPU, ID, WS, HandleCount, Threads
$Network = (Get-Counter '\Process(*)\IO Data Bytes/sec').CounterSamples.Where({$_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle"}) | sort cookedvalue -Descending | Select -Property instancename -First 10

$combinedIO = @{}                                                           # Get IO Read and Write Operations
$ioOps = Get-Counter '\Process(*)\IO Read Operations/sec', '\Process(*)\IO Write Operations/sec' |Select -ExpandProperty CounterSamples | Where-Object { $_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle" }
foreach ($op in $ioOps) {$combinedIO[$op.InstanceName] += $op.CookedValue}  # Combine and sum the IO operations for each process
$DiskIOPS = $combinedIO.GetEnumerator() | Sort-Object Value -Descending | Select-Object -First 10 -ExpandProperty Name

$sortCriteria = @('CPU', 'WS', 'HandleCount', 'Threads')                    # Using a loop to sort for each property
$sortedResults = @{}
foreach ($criteria in $sortCriteria) {$sortedResults[$criteria] = $objArr | Sort-Object $criteria -Descending | Select-Object -First 10}

$harray = for ($j = 0; $j -lt 10; $j++) {                                   # Creating the custom object array
    [PSCustomObject][ordered]@{
        'CPU'     = $sortedResults['CPU'][$j].Process 
        'WS'      = $sortedResults['WS'][$j].Process 
        'Handles' = $sortedResults['HandleCount'][$j].Process 
        'Threads' = $sortedResults['Threads'][$j].Process
        'Network' = $Network[$j].InstanceName
        'DiskIOPS' = $DiskIOPS[$j]
    }
}
$harray | Format-Table -AutoSize


CPU                     WS                      Handles                 Threads                 Network           DiskIOPS
---                     --                      -------                 -------                 -------           --------
msedge►13512            Memory Compression►2632 OneDrive►11916          System►4                code              code
OneDrive►11916          dwm►772                 System►4                Memory Compression►2632 code              dotnet
msedge►6124             Code►5344               explorer►6532           explorer►6532           code              dwm
msedge►7464             msedge►7464             msedge►7464             msedge►7464             code              msedge
Code►5344               pwsh►9916               lsass►904               SearchHost►6964         grammarly.desktop svchost
Code►16052              explorer►6532           Grammarly.Desktop►12308 Code►11620              msedge            memory comp…
Grammarly.Desktop►12308 G